In [1]:
%pip install -r requirements.txt
%pip install ipykernel langchain_experimental llama-index-vector-stores-pinecone ipykernel PyMuPDF pinecone-client pypdf faiss-cpu langchain_community transformers sentence_transformers

import fitz

import os, io, json, transformers, pinecone, pypdf, faiss, sqlite3, langchain_community, langchain, openai, math, time, nltk, torch, huggingface_hub, datasets

from openai import OpenAI

from nltk.tokenize import sent_tokenize

from transformers import pipeline

from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN

from transformers import BertTokenizer, BertModel

import pandas as pd
import numpy as np
from io import StringIO
from dotenv import load_dotenv
from operator import itemgetter

from langchain import document_loaders, embeddings

from langchain.vectorstores import FAISS
from llama_index.vector_stores.pinecone import PineconeVectorStore

from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings.openai import OpenAIEmbedding

from langchain_experimental.text_splitter import SemanticChunker

from langchain_community.vectorstores import FAISS

from llama_index.core.schema import TextNode

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from llama_index.core.node_parser import SentenceSplitter

from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from pinecone import Pinecone, ServerlessSpec, Pinecone         # vector store

from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    SimpleDirectoryReader
)

from llama_index.core.extractors import (
    QuestionsAnsweredExtractor,
    TitleExtractor,
)
from llama_index.core.ingestion import IngestionPipeline
from llama_index.llms.openai import OpenAI

from sentence_transformers import SentenceTransformer
from sentence_transformers import util

nltk.download('punkt')


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


c:\Users\hokla\anaconda3\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hokla\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
load_dotenv()

openai = os.getenv('OPENAI_API_KEY')
pinecone_api_key =os.getenv('PINECONE_API_KEY')
environment =os.getenv('PINECONE_ENV')
HF_TOKEN = os.getenv('HF_TOKEN')
pc = Pinecone(api_key=pinecone_api_key)

In [3]:
import json, os, io, re, requests, fitz
import requests
from langchain_text_splitters import RecursiveJsonSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    SimpleDirectoryReader
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core.schema import TextNode
from llama_index.core.extractors import (
    QuestionsAnsweredExtractor,
    KeywordExtractor,
    SummaryExtractor,

)


from llama_index.core.ingestion import IngestionPipeline
from llama_index.llms.openai import OpenAI

from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [4]:
load_dotenv()

openai = os.getenv('OPENAI_API_KEY')
pinecone_api_key =os.getenv('PINECONE_API_KEY')
environment =os.getenv('PINECONE_ENV')
HF_TOKEN = os.getenv('HF_TOKEN')

pc = Pinecone(api_key=pinecone_api_key)

In [5]:
doc = fitz.open(r"C:\GenAIhw3\product_info_pdf.pdf")


In [6]:
from langchain_experimental.text_splitter import SemanticChunker

text_parser = SentenceSplitter(
    chunk_size=1024
)

In [7]:
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\hokla\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [8]:
text_chunks = []
doc_idxs = []


for doc_idx, page in enumerate(doc):
    page_text = page.get_text("text")
    cur_text_chunks = text_parser.split_text(page_text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

In [9]:
nodes = []

for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc_idx = doc_idxs[idx]
    src_page = doc[src_doc_idx]
    nodes.append(node)

### **Create the vector store using chosen similarity metrics:**

In [10]:
use_serverless = os.environ.get("USE_SERVERLESS", "False").lower() == "true"

if use_serverless:
    spec = pinecone.ServerlessSpec(cloud='aws', region='us-east-1')
else:
    spec = pinecone.PodSpec(environment=environment)

index_name = "hw03"

if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

In [11]:
dimensions = 1536          

pc.create_index(
    name=index_name, 
    dimension=dimensions, 
    metric="cosine",          
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

### **choose a similarity metric to use for the vector store:**

In [12]:
for index in pc.list_indexes():
    print(index['name'])

pc.describe_index("hw03")

pc_index = pc.Index(index_name) 

vector_store = PineconeVectorStore(pinecone_index=pc_index)   

hw02
hw03


In [13]:

pc_index.describe_index_stats()


{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [14]:
llm = OpenAI(model="gpt-3.5-turbo")

extractors = [
    KeywordExtractor(keywords=5, llm=llm),  
    QuestionsAnsweredExtractor(questions=3, llm=llm), 
    SummaryExtractor(llm=llm),
]

pipeline = IngestionPipeline(
    transformations=extractors,
)
nodes = await pipeline.arun(nodes=nodes, in_place=False)

100%|██████████| 37/37 [00:09<00:00,  3.82it/s]


### ***choose an embedding model to use for the vector store:**

#### **OpenAI Embeddings**

In [15]:
model_ada="text-embedding-ada-002"
small_txt_embedmodel_="text-embedding-3-small"


In [16]:
embed_model = OpenAIEmbedding(model="text-embedding-ada-002", openai_api_key=openai)

for node in nodes:
    actual_text = node.get_content(metadata_mode="all") 
    
    node_embedding = embed_model.get_text_embedding(actual_text)
    
    node.embedding = node_embedding
    
    node.metadata["text_content"] = actual_text

### **load the embeddings into the vector store (e.g. create a vector store):**

In [17]:
vector_store.add(nodes)

Upserted vectors: 100%|██████████| 37/37 [00:00<00:00, 56.15it/s]


['7cda20c2-729a-4de4-9e49-b840a41b0eff',
 '731f9c30-c832-46f3-8f1b-cf56ed3826d4',
 'ec06adcd-0ceb-4578-b666-2b3aa7c10a4e',
 'c8563d7a-fc8c-45c2-8063-01c7f5d76f41',
 '987d3dd2-226d-4350-9f47-92b3a6958fb9',
 '272c7c6f-39b5-4f34-9281-b6cfd4076baf',
 '1c232e93-9bd1-49b0-a718-c2e07d3d47e0',
 'cb8f2ac7-f5af-4912-9dbe-d7d33aee521d',
 '79a6b4d1-48a9-42cc-8e68-0e1cf18c8ebb',
 '648bcc9e-b63b-483b-8918-61ba23bdcb6e',
 '78a85337-66cd-4c94-a9ec-b0ceb0df7142',
 '89301c0f-0824-4674-9016-60e2902d8a43',
 '09f98389-0892-4642-aa7e-c2346e0475d0',
 'bd345d6f-44ad-4347-8ec1-2f0ae1529ace',
 'c57d3601-e485-470e-9a5f-bea48a8754dd',
 '8a8f5373-9e30-457c-a337-204dc9aa793b',
 '9845de5f-b1e5-4df2-baf1-2fe6efcf0e16',
 '8fbb6666-c0e2-4bd9-8238-2578bee707f4',
 '3abcffdc-f1be-4e60-bc62-6461504c17c4',
 '7bdbd7f5-e4e5-4afc-84a9-772e7a1c6e77',
 '679b552b-2ccf-48f5-a147-9f6704103cb5',
 '5df5bf44-ee43-4db0-ae3f-821ec41d42c5',
 '9a8359c2-c0cd-47e2-b6d5-b0dbe37df69c',
 '315c5565-818b-4dd8-b69c-7c546e84919c',
 '5a9e7041-eb01-

In [18]:
pc_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [19]:
print(nodes[0].metadata)

{'excerpt_keywords': 'Breville, Espresso Machine, Specialty Coffee, Grinder, Latte Art', 'questions_this_excerpt_can_answer': '1. How does the Breville BES870XL Espresso Machine contribute to the third wave specialty coffee experience at home?\n2. What specific features of the Breville Barista Express make it stand out as an all-in-one espresso machine with an integrated grinder?\n3. What accessories are included with the Breville BES870XL Espresso Machine, and how do they enhance the overall coffee-making experience for users?', 'section_summary': 'The section discusses the Breville BES870XL Espresso Machine, highlighting its features such as dose control grinding, optimal water pressure, precise espresso extraction, and manual microfoam milk texturing for creating latte art. It also mentions the included accessories, capacity, settings, and warranty of the machine. The section emphasizes how the Breville Barista Express contributes to the third wave specialty coffee experience at hom

In [20]:
print(nodes[0])

Node ID: 7cda20c2-729a-4de4-9e49-b840a41b0eff
Text: 94-844: Generative AI Labs    Product Information    1.
Category: Home and Kitchen    We chose this category because it is the
most popular, as well as the highest  contributing category on Amazon.
Product: Breville BES870XL Espresso Machine, One Size, Brushed
Stainless Steel    Description:  About this item  The Breville Barista
Express deli...


### **Retrieve Content from the Vector Store**

In [21]:
from openai import OpenAI
client = OpenAI()

## General Prompt & Result

#### Coffee Machine

In [ ]:
query1 = "Find information specifically about Breville espresso machiness only."

query_response = client.embeddings.create(
    model="text-embedding-ada-002",
    input=[query1]
)

query_embedding = query_response.data[0].embedding

res2 = pc_index.query(
    vector=query_embedding, 
    top_k=5, 
    include_metadata=True
)

retrieved_texts = []
for match in res2['matches']:
    actual_text = match['metadata'].get('text_content', 'No text available')
    retrieved_texts.append(actual_text)

combined_context = "\n\n".join(retrieved_texts)

def rag_openai_gpt(model, query, retrieved_context):
    """
    Reason over the retrieved context using OpenAI's GPT model.
    """
    messages = [
        {"role": "system", "content": "Use the provided information to answer the user's question."},
        {"role": "user", "content": f"The question is: {query}"},
        {"role": "assistant", "content": f"The retrieved information is:\n\n{retrieved_context}"},
        {"role": "user", "content": "What is the answer?"}
    ]
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return "Error: Unable to generate an answer."

query_to_answer = "Provide me a more wholistic understanding (description of its look, feature, and possible failure) of Breville espresso machiness"
model_to_use = "gpt-4"
final_answer = rag_openai_gpt(model_to_use, query_to_answer, combined_context)

print("Final Answer:")
print(final_answer)


Final Answer:
Apologies for the confusion, but the provided information does not include any specific details on the Breville espresso machines. Please provide information about the look, feature, and possible failure of Breville espresso machines.


#### Meta Quest

In [ ]:
query1 = "Find information specifically about Meta Quest only."

query_response = client.embeddings.create(
    model="text-embedding-ada-002",
    input=[query1]
)

query_embedding = query_response.data[0].embedding

res2 = pc_index.query(
    vector=query_embedding, 
    top_k=5, 
    include_metadata=True
)

retrieved_texts = []
for match in res2['matches']:
    actual_text = match['metadata'].get('text_content', 'No text available')
    retrieved_texts.append(actual_text)

combined_context = "\n\n".join(retrieved_texts)

def rag_openai_gpt(model, query, retrieved_context):
    """
    Reason over the retrieved context using OpenAI's GPT model.
    """
    messages = [
        {"role": "system", "content": "Use the provided information to answer the user's question."},
        {"role": "user", "content": f"The question is: {query}"},
        {"role": "assistant", "content": f"The retrieved information is:\n\n{retrieved_context}"},
        {"role": "user", "content": "What is the answer?"}
    ]
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return "Error: Unable to generate an answer."

query_to_answer = "Provide me a more wholistic understanding (description of its look, feature, and possible failure) of Meta Quest 3"
model_to_use = "gpt-4"  
final_answer = rag_openai_gpt(model_to_use, query_to_answer, combined_context)

print("Final Answer:")
print(final_answer)


Final Answer:
I'm sorry, but I can't provide the information you're looking for because Meta Quest 3 hasn't been released yet. All details about its look, features, and possible failures remain unannounced to the public. The current latest VR headset from Meta (previously Facebook) is Meta Quest 2. Let's keep an eye out for updates from the official Meta platforms for the accurate information.


#### Handpan Drum

In [ ]:
query1 = "Find information specifically about Handpan Drum only."

query_response = client.embeddings.create(
    model="text-embedding-ada-002",
    input=[query1]
)

query_embedding = query_response.data[0].embedding

res2 = pc_index.query(
    vector=query_embedding, 
    top_k=10, 
    include_metadata=True
)

retrieved_texts = []
for match in res2['matches']:
    actual_text = match['metadata'].get('text_content', 'No text available')
    retrieved_texts.append(actual_text)

combined_context = "\n\n".join(retrieved_texts)

def rag_openai_gpt(model, query, retrieved_context):
    """
    Reason over the retrieved context using OpenAI's GPT model.
    """
    messages = [
        {"role": "system", "content": "Use the provided information to answer the user's question."},
        {"role": "user", "content": f"The question is: {query}"},
        {"role": "assistant", "content": f"The retrieved information is:\n\n{retrieved_context}"},
        {"role": "user", "content": "What is the answer?"}
    ]
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return "Error: Unable to generate an answer."

query_to_answer = "Provide me a more wholistic understanding (description of its look, feature, and possible failure) of Handpan Drum"
model_to_use = "gpt-4"
final_answer = rag_openai_gpt(model_to_use, query_to_answer, combined_context)

print("Final Answer:")
print(final_answer)


Final Answer:
A Handpan Drum is a captivating instrument known for its ethereal, resonant sound and unique design. Resembling a UFO, its structure is typically created from two dome-shaped pieces of steel glued together with a circular, hollow interior. The rounded top - called the 'ding' - features several dents or impressions, each of which represents a different musical note. The other side, or 'gu', contains a hollow opening that affects the instrument's reverberations.

The features of this instrument include its portability due to its relatively small size and light weight, often measuring between 20-24 inches in diameter at most. It also emits a rich, warm and sound that's a combination of rhythm, melody and harmony all in one instrument.

In terms of possible failures, like any musical instrument, the Handpan Drum requires careful maintenance to prevent issues. Overplaying can lead to note detuning. As such, regular tunings by a professional may be necessary. Also, due to its s

## Visual Information & Result

#### Coffee Machine

In [24]:
query1 = "Find information specifically about Breville espresso machiness only."

query_response = client.embeddings.create(
    model="text-embedding-ada-002",
    input=[query1]
)

query_embedding = query_response.data[0].embedding

res2 = pc_index.query(
    vector=query_embedding, 
    top_k=5, 
    include_metadata=True
)

retrieved_texts = []
for match in res2['matches']:
    actual_text = match['metadata'].get('text_content', 'No text available')
    retrieved_texts.append(actual_text)

combined_context = "\n\n".join(retrieved_texts)

def rag_openai_gpt(model, query, retrieved_context):
    """
    Reason over the retrieved context using OpenAI's GPT model.
    """
    messages = [
        {"role": "system", "content": "Use the provided information to answer the user's question."},
        {"role": "user", "content": f"The question is: {query}"},
        {"role": "assistant", "content": f"The retrieved information is:\n\n{retrieved_context}"},
        {"role": "user", "content": "What is the answer?"}
    ]
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return "Error: Unable to generate an answer."

query_to_answer = "Provide me some bullet points of visual informations of Breville espresso machiness that helps generating product image"
model_to_use = "gpt-4"
final_answer = rag_openai_gpt(model_to_use, query_to_answer, combined_context)

print("Final Answer:")
print(final_answer)


Final Answer:
Key visual details about Breville espresso machines that aid in generating the product image include:

- The Breville espresso machines are well-designed products with thoughtful details and sturdy construction, visually appealing materials, and a compact size that can fit into any kitchen environment.
- The products have high-quality attached features like a magnetized hole for tamper storage, a hidden accessories tray in the base of the machine, and an area on the machine's top surface that stays warm from the boiler to keep your portafilter and coffee cup warm.
- Breville Barista Express offers all-in-one espresso machines with an integrated grinder that goes from beans to espresso in under one minute. 
- The machines have a conical burr grinder, a simple and intuitive grind size dial, and a direct grind-to-portafilter option.
- The Breville espresso machines come with manual microfoam milk texturing that enhance flavor and enables creation of latte art.
- The products

#### Meta Quest

In [26]:
query1 = "Find information specifically about Meta Quest only."

query_response = client.embeddings.create(
    model="text-embedding-ada-002",
    input=[query1]
)

query_embedding = query_response.data[0].embedding

res2 = pc_index.query(
    vector=query_embedding, 
    top_k=5, 
    include_metadata=True
)

retrieved_texts = []
for match in res2['matches']:
    actual_text = match['metadata'].get('text_content', 'No text available')
    retrieved_texts.append(actual_text)

combined_context = "\n\n".join(retrieved_texts)

def rag_openai_gpt(model, query, retrieved_context):
    """
    Reason over the retrieved context using OpenAI's GPT model.
    """
    messages = [
        {"role": "system", "content": "Use the provided information to answer the user's question."},
        {"role": "user", "content": f"The question is: {query}"},
        {"role": "assistant", "content": f"The retrieved information is:\n\n{retrieved_context}"},
        {"role": "user", "content": "What is the answer?"}
    ]
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return "Error: Unable to generate an answer."

query_to_answer = "Provide me some bullet points of visual information of Meta Quest 3 that helps generating product image"
model_to_use = "gpt-4"
final_answer = rag_openai_gpt(model_to_use, query_to_answer, combined_context)

print("Final Answer:")
print(final_answer)


Final Answer:
Visual Information on the Meta Quest 3 VR headset includes:

- Offers immersive mixed reality experiences.
- Comes with a 4K + Infinite Display feature contributing to a lifelike gaming experience.
- Comes preloaded with exclusive content like Batman: Arkham Shadow.
- Offers more storage for apps and games.
- Device creates a wider, taller, expansive field of view for the gamers.
- Resolution per eye is recorded to be 2064x2208.
- Has an improved room scanning tool for a smooth gaming experience.
- Can perform exceptionally well with a gaming PC.
- Comes with an option to stream games.
- Battery life varies depending on the use. However, using Augmented Reality can cut the battery life into half.
- Possible issues with the Quest Link software, causing difficulty in connecting the headset to PC.
- The default strap is considered uncomfortable by some users. Opting for a third-party strap could be more beneficial.
- Visual clarity is high compared to other VR headsets like 

#### Handpan Drum

In [25]:
query1 = "Find information specifically about Handpan Drum only."

query_response = client.embeddings.create(
    model="text-embedding-ada-002",
    input=[query1]
)

query_embedding = query_response.data[0].embedding

res2 = pc_index.query(
    vector=query_embedding, 
    top_k=5, 
    include_metadata=True
)

retrieved_texts = []
for match in res2['matches']:
    actual_text = match['metadata'].get('text_content', 'No text available')
    retrieved_texts.append(actual_text)

combined_context = "\n\n".join(retrieved_texts)

def rag_openai_gpt(model, query, retrieved_context):
    """
    Reason over the retrieved context using OpenAI's GPT model.
    """
    messages = [
        {"role": "system", "content": "Use the provided information to answer the user's question."},
        {"role": "user", "content": f"The question is: {query}"},
        {"role": "assistant", "content": f"The retrieved information is:\n\n{retrieved_context}"},
        {"role": "user", "content": "What is the answer?"}
    ]
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return "Error: Unable to generate an answer."

query_to_answer = "Provide me some bullet points of visual information of Handpan Drum that helps generating product image"
model_to_use = "gpt-4"
final_answer = rag_openai_gpt(model_to_use, query_to_answer, combined_context)

print("Final Answer:")
print(final_answer)

Final Answer:
- The Handpan Drum is crafted from premium 1.2mm thick steel using advanced high-temperature heat treatment techniques for durability, robustness, and resistance to rust.
- Each drum undergoes expert manual tuning to ensure precise tones and harmonies, resulting in unmatched accuracy and resonance.
- The sound quality of the Handpan Drum is described as clean, ethereal, and captivating, purifying the listener's soul and making it suitable for music education, yoga meditation, and musical performances.
- The Handpan Drum is easy to play and includes 10 notes in D minor (A3, Bb3, C4, D3, D4, E4, F4, G4, A4, C5). No other equipment is necessary.
- Some HandPans may have differences in tuning, with some being closer to A = 440 Hz rather than the advertised A = 432 Hz.
- Depending on the model, the Handpan Drum may come in different colors like black or gold. Sizes can range from 14 inches to 22 inches.
- Each Handpan Drum purchase comes with a complete package including a car

## Json File Format

#### Breville espresso machiness

In [27]:
query1 = "Find information specifically about visual information of Breville espresso machiness only."

query_response = client.embeddings.create(
    model="text-embedding-ada-002",
    input=[query1]
)

query_embedding = query_response.data[0].embedding

res2 = pc_index.query(
    vector=query_embedding, 
    top_k=5, 
    include_metadata=True
)

retrieved_texts = []
for match in res2['matches']:
    actual_text = match['metadata'].get('text_content', 'No text available')
    retrieved_texts.append(actual_text)

combined_context = "\n\n".join(retrieved_texts)

def rag_openai_gpt(model, query, retrieved_context):
    """
    Reason over the retrieved context using OpenAI's GPT model.
    """
    messages = [
        {"role": "system", "content": "Use the provided information to answer the user's question."},
        {"role": "user", "content": f"The question is: {query}"},
        {"role": "assistant", "content": f"The retrieved information is:\n\n{retrieved_context}"},
        {"role": "user", "content": "Using the information provided, generate a structured JSON file containing details of the product"}
    ]
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return "Error: Unable to generate an answer."

query_to_answer = "Provide me bullet point info of Breville espresso machines that could help design its product image. The JSON should include attributes like size, colors, material, notable features, etc. Ensure the response is strictly in JSON format without any explanations."
model_to_use = "gpt-4"  
final_answer = rag_openai_gpt(model_to_use, query_to_answer, combined_context)

print("Final Answer:")
print(final_answer)

Final Answer:
{
  "Product_Details": {
    "Product": "Breville BES870XL Espresso Machine",
    "Size": "One Size",
    "Colors": "Brushed Stainless Steel",
    "Material": "Stainless Steel",
    "Notable_Features":
    [
      "High-quality design",
      "Consistent espresso production",
      "Quick water boiler heating",
      "High-quality milk foam production",
      "In-built conical burr grinder",
      "Precision dose control grinding",
      "Optimal water pressure control",
      "Precise espresso extraction with digital temperature control",
      "Manual micro foam milk texturing",
      "User-friendly control and movable parts",
      "Integrated tamper",
      "Mass: 1/2 lb Bean Hopper; 67 oz Water Tank",
      "Settings: Single or double shots; Adjustable Grind Amount and Manual Override",
      "Power and Voltage: 1600 Watts; 120 Volts",
      "Useful for home baristas and beginners"
    ],
    "Accessories_Included":
    [
      "Razor Dose Trimming Tool",
      "54mm

In [29]:
query1 = "Find information specifically about visual information of Meta Quest only."

query_response = client.embeddings.create(
    model="text-embedding-ada-002",
    input=[query1]
)

query_embedding = query_response.data[0].embedding

res2 = pc_index.query(
    vector=query_embedding, 
    top_k=5, 
    include_metadata=True
)

retrieved_texts = []
for match in res2['matches']:
    actual_text = match['metadata'].get('text_content', 'No text available')
    retrieved_texts.append(actual_text)

combined_context = "\n\n".join(retrieved_texts)

def rag_openai_gpt(model, query, retrieved_context):
    """
    Reason over the retrieved context using OpenAI's GPT model.
    """
    messages = [
        {"role": "system", "content": "Use the provided information to answer the user's question."},
        {"role": "user", "content": f"The question is: {query}"},
        {"role": "assistant", "content": f"The retrieved information is:\n\n{retrieved_context}"},
        {"role": "user", "content": "Using the information provided, generate a structured JSON file containing details of the product"}
    ]
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return "Error: Unable to generate an answer."

query_to_answer = "Provide me bullet point info of Meta Quest 3 that could help design its product image. The JSON should include attributes like size, colors, material, notable features, etc. Ensure the response is strictly in JSON format without any explanations."
model_to_use = "gpt-4"  
final_answer = rag_openai_gpt(model_to_use, query_to_answer, combined_context)

print("Final Answer:")
print(final_answer)

Final Answer:
{
  "Product_Name": "Meta Quest 3 512GB",
  "Product_Details": {
    "Size": "512 GB",
    "Style": "Headset + Controllers",
    "Resolution": "4K + Infinite Display (2064x2208 resolution per eye)",
    "Sound": "Rich spatial audio",
    "Material": "Not specified",
    "Colors": "Not specified",
    "Features": {
      "Mixed Reality Capability": "Ability to blend digital objects into physical space",
      "4K + Infinite Display": "Offers wide, tall, expansive field of view",
      "Premium Gaming Experience": "Improved resolution, crisper visuals, reduced screen door effect compared to predecessor models",
      "Room Scanning Tool": "Improved tool for virtual room creation using multiple cameras",
      "Battery": "Faster consumption especially in augmented reality mode, battery pack or charging cable recommended for prolonged use",
      "Extra Content": "Inclusive of Batman: Arkham Shadow and a 3-month trial of Meta Quest+",
      "Wireless Capability": "High perfor

In [28]:
query1 = "Find information specifically about visual information of Handpan Drum only."

query_response = client.embeddings.create(
    model="text-embedding-ada-002",
    input=[query1]
)

query_embedding = query_response.data[0].embedding

res2 = pc_index.query(
    vector=query_embedding, 
    top_k=5, 
    include_metadata=True
)

retrieved_texts = []
for match in res2['matches']:
    actual_text = match['metadata'].get('text_content', 'No text available')
    retrieved_texts.append(actual_text)

combined_context = "\n\n".join(retrieved_texts)

def rag_openai_gpt(model, query, retrieved_context):
    """
    Reason over the retrieved context using OpenAI's GPT model.
    """
    messages = [
        {"role": "system", "content": "Use the provided information to answer the user's question."},
        {"role": "user", "content": f"The question is: {query}"},
        {"role": "assistant", "content": f"The retrieved information is:\n\n{retrieved_context}"},
        {"role": "user", "content": "Using the information provided, generate a structured JSON file containing details of the product"}
    ]
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return "Error: Unable to generate an answer."

query_to_answer = "Provide me bullet point info of Handpan Drum that could help design its product image. The JSON should include attributes like size, colors, material, notable features, etc. Ensure the response is strictly in JSON format without any explanations."
model_to_use = "gpt-4"  
final_answer = rag_openai_gpt(model_to_use, query_to_answer, combined_context)

print("Final Answer:")
print(final_answer)

Final Answer:
{
  "Handpan Drum": {
    "Size": [
      "22 Inches",
      "14 Inches"
    ],
    "Color": [
      "Gold",
      "Black",
      "Purple",
      "Print Black 10 Sounds"
    ],
    "Material": "Premium Steel",
    "Notable Features": {
      "Tuning": "D Minor Kurd, 432Hz, 10 Notes",
      "Ease of Use": "Easy to pick up without any other equipment",
      "Includes Accessories": "Velvet bag, two 'drum sticks', small booklet of songs",
      "Natural Woven Decoration": "Yes",
      "Sound Quality": "Sound is clear with the exception of bass notes, which can sound flat",
      "Packed Well": "Yes"
    },
    "Customer Reviews": [
      {
        "Rating": "5.0 out of 5 stars",
        "Review Date": "October 23, 2024",
        "Review Text": "Excellent, definitely would upgrade soon"
      },
      {
        "Rating": "4.0 out of 5 stars",
        "Review Date": "July 6, 2024",
        "Review Text": "Good instrument, tuning is not A = 432 Hz."
      },
      {
        "Ra